In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
data_dir = "inaturalist_12K"

In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

WindowsPath('inaturalist_12K')

In [4]:
image_count = len(list(data_dir.glob('train/Fungi/*.jpg')))
print(image_count)

999


In [5]:
data_images_dict = {
    'amphibia': list(data_dir.glob('train/Amphibia/*')),
    'animalia': list(data_dir.glob('train/Animalia/*')),
    'arachnida': list(data_dir.glob('train/Arachnida/*')),
    'aves': list(data_dir.glob('train/Aves/*')),
    'fungi': list(data_dir.glob('train/Fungi/*')),
    'insecta': list(data_dir.glob('train/Insecta/*')),
    'mammalia': list(data_dir.glob('train/Mammalia/*')),
    'molusca': list(data_dir.glob('train/Mollusca/*')),
    'plantae': list(data_dir.glob('train/Plantae/*')),
    'reptilia': list(data_dir.glob('train/Reptilia/*')),
}

In [6]:
data_labels_dict = {
    'amphibia': 0,
    'animalia': 1,
    'arachnida': 2,
    'aves': 3,
    'fungi': 4,
    'insecta': 5,
    'mammalia': 6,
    'molusca': 7,
    'plantae': 8,
    'reptilia': 9,

}

In [7]:
img = cv2.imread(str(data_images_dict['aves'][2]))
img.shape


(601, 800, 3)

In [ ]:
X_train, y_train = [], []

for data_name, images in data_images_dict.items():
    for image in images:
        print(str(image))
        if str(image)=="inaturalist_12K\\train\\Fungi\\.DS_Store":
          continue
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X_train.append(resized_img)
        y_train.append(data_labels_dict[data_name])

In [9]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [10]:
data_images_dict_test = {
    'amphibia': list(data_dir.glob('val/Amphibia/*')),
    'animalia': list(data_dir.glob('val/Animalia/*')),
    'arachnida': list(data_dir.glob('val/Arachnida/*')),
    'aves': list(data_dir.glob('val/Aves/*')),
    'fungi': list(data_dir.glob('val/Fungi/*')),
    'insecta': list(data_dir.glob('val/Insecta/*')),
    'mammalia': list(data_dir.glob('val/Mammalia/*')),
    'molusca': list(data_dir.glob('val/Mollusca/*')),
    'plantae': list(data_dir.glob('val/Plantae/*')),
    'reptilia': list(data_dir.glob('val/Reptilia/*')),
}

In [ ]:
X_test, y_test = [], []

for data_name, images in data_images_dict_test.items():
    for image in images:
        print(str(image))
        # if str(image)=="/content/inaturalist_12K/train/Fungi/.DS_Store":
        #   continue
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X_test.append(resized_img)
        y_test.append(data_labels_dict[data_name])

In [12]:
X_test = np.array(X_test)
y_test = np.array(y_test)

In [13]:
X_train_scaled = []
for x_train in X_train:
    x_train_scaled = x_train / 255
    X_train_scaled.append(x_train_scaled)
X_train_scaled = np.array(X_train_scaled)
X_test_scaled = []
for x_test in X_test:
    x_test_scaled = x_test / 255
    X_test_scaled.append(x_test_scaled)
X_test_scaled = np.array(X_test_scaled)
# X_train_scaled = X_train / 255
# X_test_scaled = X_test / 255

## Now take pre-trained model and retrain it using given images

In [14]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [15]:
num_of_classes = 10

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 10)                12810     
                                                                 
Total params: 2270794 (8.66 MB)
Trainable params: 12810 (50.04 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [16]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=30)

Epoch 1/30
313/313 [==============================] - 178s 557ms/step - loss: 1.1379 - acc: 0.6239
Epoch 2/30
313/313 [==============================] - 171s 545ms/step - loss: 0.8342 - acc: 0.7272
Epoch 3/30
313/313 [==============================] - 170s 542ms/step - loss: 0.7445 - acc: 0.7532
Epoch 4/30
313/313 [==============================] - 165s 526ms/step - loss: 0.6906 - acc: 0.7725
Epoch 5/30
313/313 [==============================] - 164s 523ms/step - loss: 0.6455 - acc: 0.7865
Epoch 6/30
313/313 [==============================] - 167s 533ms/step - loss: 0.6076 - acc: 0.7990
Epoch 7/30
313/313 [==============================] - 168s 537ms/step - loss: 0.5784 - acc: 0.8087
Epoch 8/30
313/313 [==============================] - 168s 538ms/step - loss: 0.5521 - acc: 0.8194
Epoch 9/30
313/313 [==============================] - 169s 541ms/step - loss: 0.5299 - acc: 0.8264
Epoch 10/30
313/313 [==============================] - 168s 538ms/step - loss: 0.5065 - acc: 0.8343
Epoch 11/

In [17]:
model.evaluate(X_test_scaled,y_test)

63/63 [==============================] - 32s 499ms/step - loss: 1.1136 - acc: 0.7045


[1.1135525703430176, 0.7045000195503235]

In [18]:
model.save('Weights/naturalist_30.keras')

In [19]:

model2 = keras.models.load_model('Weights/naturalist_30.keras')

TypeError: The added layer must be an instance of class Layer. Received: layer=KerasLayer of type <class 'str'>.

In [ ]:
drop_model = models.Sequential()
drop_model.add(layers.Dense(64, activation='relu', input_shape=(NB_WORDS,)))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(64, activation='relu'))
drop_model.add(layers.Dropout(0.5))
drop_model.add(layers.Dense(3, activation='softmax'))
drop_model.name = 'Dropout layers model'
drop_history = deep_model(drop_model, X_train_rest, y_train_rest, X_valid, y_valid)
drop_min = optimal_epoch(drop_history)
eval_metric(drop_model, drop_history, 'loss')